In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/uci-secom.csv


In [2]:
filename = "/kaggle/input/uci-secom.csv"

semi = pd.read_csv(filename)
semi.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [3]:
semi.isnull().sum().describe()

count     592.000000
mean       70.863176
std       241.476304
min         0.000000
25%         2.000000
50%         6.000000
75%         9.000000
max      1429.000000
dtype: float64

In [4]:
columns_to_remove = []
j = semi.isnull().sum()
for i in j.keys():
    if j[i] >= 900:
        print(i, j[i])
        columns_to_remove.append(i)
        
data = semi.drop(columns_to_remove, axis = 1)

85 1341
109 1018
110 1018
111 1018
157 1429
158 1429
220 1341
244 1018
245 1018
246 1018
292 1429
293 1429
358 1341
382 1018
383 1018
384 1018
492 1341
516 1018
517 1018
518 1018
578 949
579 949
580 949
581 949


In [5]:
from sklearn.impute import SimpleImputer 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, average_precision_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [6]:
x = data.iloc[:, 1:-1]
y = data.iloc[:, -1]

imp = SimpleImputer()
x = imp.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 10)

In [7]:
print(x_train.shape)
print(x_test.shape)
print(data.shape)

(1253, 566)
(314, 566)
(1567, 568)


In [8]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)
## logistic regression, regularized


Cs = [(i+1) for i in range(10)]
f1 = []

for c in Cs:
    lr = LogisticRegression(C = c, solver = 'lbfgs', max_iter=400, penalty = 'l2')
    f1 = cross_val_score(lr, x_train_std, y_train, cv=5, scoring='f1_macro')
    print(c)
    print(f1.mean())

1
0.5611551815478494
2
0.5551798045616154
3
0.5591687017918965
4
0.5580584106412115
5
0.5524621568777687
6
0.5515342260870288
7
0.5511057256434826
8
0.5543239996800593
9
0.5539355133385903
10
0.5535546188070658


In [9]:
## logistic regression, regularized

## based on CV, C = 1
c = 1

lr = LogisticRegression(C = c, solver = 'lbfgs', max_iter=400, penalty = 'l2')
lr.fit(x_train_std, y_train)
y_pred_lr = lr.predict(x_test_std)
print('accuracy score', accuracy_score(y_test, y_pred_lr))
y_pred_lr = lr.predict(x_train_std)
cm = confusion_matrix(y_train, y_pred_lr)
print('confusion matrix_train {}'.format(cm))
y_pred_lr = lr.predict(x_test_std)
cm = confusion_matrix(y_test, y_pred_lr)
print('confusion matrix_test {}'.format(cm))
print('precision_score', average_precision_score(y_test, y_pred_lr))
print('recall_score', recall_score(y_test, y_pred_lr))
print('f1_score', f1_score(y_test, y_pred_lr))
#f1.append(f1_score(y_test, y_pred_lr))

#index = [i for i in range(len(Cs))]
#plt.plot(index, f1, 'bs')

accuracy score 0.9076433121019108
confusion matrix_train [[1170    0]
 [  12   71]]
confusion matrix_test [[278  15]
 [ 14   7]]
precision_score 0.15064659332175256
recall_score 0.3333333333333333
f1_score 0.3255813953488372


In [10]:
## svm, regularized
from sklearn.svm import SVC

Cs = [(i+1)/10 for i in range(10)]
f1 = []

for c in Cs:
    clf_svm = SVC(C = c, kernel='linear', gamma='scale')
    print(c)
    f1 = cross_val_score(clf_svm, x_train_std, y_train, cv=5, scoring='f1_macro')
    print(f1.mean())

0.1
0.5379325627293214
0.2
0.5444094989052353
0.3
0.5393532249230871
0.4
0.5488260068559142
0.5
0.5551895200727436
0.6
0.5569290789392645
0.7
0.5569290789392645
0.8
0.5569290789392645
0.9
0.5569290789392645
1.0
0.5569290789392645


In [11]:
# check models on test data
clf_svm = SVC(C = 0.6, kernel='linear', degree=1, gamma='scale')
clf_svm.fit(x_train_std, y_train)
y_test_pred = clf_svm.predict(x_test_std)
print('confusion_matrix', confusion_matrix(y_test, y_test_pred))
print('f1_score',f1_score(y_test, y_test_pred))

confusion_matrix [[272  21]
 [ 12   9]]
f1_score 0.3529411764705882


In [12]:
## logistic regression and SVM show similar accuracy. SVM is slightly better